In [ ]:
!pip install tweet-preprocessor
!pip install textblob
!pip install sastrawi
!pip install emoji
!pip install PySastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 4.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=f07922a29a0bd92fde9198925366e79c342d8a18ea875072cf403f698e515a95
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 4.0 MB/s 


# **SCRAPING DATA**

In [ ]:
api_key = "bTmeGgcG9rgXj8zO2YWFXhr9T"
api_secret_key = "iV09vX4XNcAZjV7tHvEupN36bN85Ca1gi1WdrtBHyUGf72EYp4"
acces_token = "1255536803009884160-ctMf7m6JwRi8RLqIq5fccbgASdUjB9"
acces_token_secret = "6w2I8i7GbSoIGt86kAEOyUER85qX5aBjF6QbrddaLVJjV"

In [ ]:
import tweepy
import re
import os
import pandas as pd
import string
from textblob import TextBlob
import preprocessor as p
from preprocessor.api import clean, tokenize, parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import datetime
from datetime import timedelta
import numpy as np
import emoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import wordcloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory



In [ ]:
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(acces_token, acces_token_secret)
api = tweepy.API(auth)

In [ ]:
def scraptweets(search_word):

  db_tweets = pd.DataFrame(columns=['username','tweetcreatedts','text'])
  tweets = tweepy.Cursor(
      api.search, q=search_word, lang='id', tweet_mode='extended').items(500)

  tweet_list = [tweet for tweet in tweets]

  for tweet in tweet_list :
    username = tweet.user.screen_name
    tweetcreatedts = tweet.created_at

    try:
      text = tweet.retweeted_status.full_text
    except AttributeError:
      text = tweet.full_text

    ith_tweet = [username, tweetcreatedts, text]

    db_tweets.loc[len(db_tweets)] = ith_tweet
    print('jumlah data : ', len(db_tweets))
  filename = '8.csv'
  db_tweets.to_csv(filename, index=False)


search_words = 'berbeda sosial'


scraptweets(search_words)


jumlah data :  1
jumlah data :  2
jumlah data :  3
jumlah data :  4
jumlah data :  5
jumlah data :  6
jumlah data :  7
jumlah data :  8
jumlah data :  9
jumlah data :  10
jumlah data :  11
jumlah data :  12
jumlah data :  13
jumlah data :  14
jumlah data :  15
jumlah data :  16
jumlah data :  17
jumlah data :  18
jumlah data :  19
jumlah data :  20
jumlah data :  21
jumlah data :  22
jumlah data :  23
jumlah data :  24
jumlah data :  25
jumlah data :  26
jumlah data :  27
jumlah data :  28
jumlah data :  29
jumlah data :  30
jumlah data :  31
jumlah data :  32
jumlah data :  33
jumlah data :  34
jumlah data :  35
jumlah data :  36
jumlah data :  37
jumlah data :  38
jumlah data :  39
jumlah data :  40
jumlah data :  41
jumlah data :  42
jumlah data :  43
jumlah data :  44
jumlah data :  45
jumlah data :  46
jumlah data :  47
jumlah data :  48
jumlah data :  49
jumlah data :  50
jumlah data :  51
jumlah data :  52
jumlah data :  53
jumlah data :  54
jumlah data :  55
jumlah data :  56
j

In [ ]:

data2 = pd.read_csv('/content/1.csv')
data3 = pd.read_csv('/content/2.csv')
data4 = pd.read_csv('/content/3.csv')
data5 = pd.read_csv('/content/4.csv')
data6 = pd.read_csv('/content/5.csv')
data7 = pd.read_csv('/content/6.csv')
data8 = pd.read_csv('/content/8.csv')


In [ ]:
frames = [data2, data3, data4, data5, data6, data7,data8]
result = pd.concat(frames).drop_duplicates().reset_index(drop=True)

In [ ]:
result.shape

(1060, 3)

In [ ]:
result.head()

,username,tweetcreatedts,text
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat..."
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun..."
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ..."


In [ ]:
result.to_csv(r'data kotor.csv', index=False)

# **DATA PREPROCESSING**

In [ ]:
tweet_df = result

In [ ]:
#cleaning

def remove_pattern(text, patern_regex):
  r = re.findall(patern_regex, text)
  for i in r:
    text = re.sub(i, '', text)
  return text

In [ ]:
tweet_df['clean_tweet']= np.vectorize(remove_pattern)(tweet_df['text'], " *RT* | *@[\w]*")
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat..."
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks..."
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ...","Sejak Negara ini belum berdiri, Perbedaan sdh ..."


In [ ]:
# remove simbol
def remove(text):
  text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
  return text
tweet_df['remove_http']= tweet_df['clean_tweet'].apply(lambda x: remove(x))
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat...",SobatDamai Toleransi menjadi dasar yang kuat u...
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks...",Gas bu jangan dibiarin apapun alasanya maksa i...
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ...","Sejak Negara ini belum berdiri, Perbedaan sdh ...",Sejak Negara ini belum berdiri Perbedaan sdh m...


In [ ]:
def remov(tweet):
  tweet = re.sub(r'\$\w','',tweet)

  tweet = re.sub(r'^RT[\s]+','',tweet)

  tweet = re.sub(r'#','',tweet)

  tweet = re.sub(r'[0-9]','',tweet)

  return tweet

tweet_df['remove_hastag'] = tweet_df['remove_http'].apply(lambda x: remov(x))
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat...",SobatDamai Toleransi menjadi dasar yang kuat u...,SobatDamai Toleransi menjadi dasar yang kuat u...
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks...",Gas bu jangan dibiarin apapun alasanya maksa i...,Gas bu jangan dibiarin apapun alasanya maksa i...
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ...","Sejak Negara ini belum berdiri, Perbedaan sdh ...",Sejak Negara ini belum berdiri Perbedaan sdh m...,Sejak Negara ini belum berdiri Perbedaan sdh m...


In [ ]:
#menghapus tweet yang sama atau retweet

tweet_df.drop_duplicates(subset = 'remove_hastag', keep = 'first', inplace = True)
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat...",SobatDamai Toleransi menjadi dasar yang kuat u...,SobatDamai Toleransi menjadi dasar yang kuat u...
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks...",Gas bu jangan dibiarin apapun alasanya maksa i...,Gas bu jangan dibiarin apapun alasanya maksa i...
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ...","Sejak Negara ini belum berdiri, Perbedaan sdh ...",Sejak Negara ini belum berdiri Perbedaan sdh m...,Sejak Negara ini belum berdiri Perbedaan sdh m...


In [ ]:
#case folding, tokenizing, filtering/stopword dan stemming
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

stop_factory = StopWordRemoverFactory().get_stop_words()
more_stopwords = ['yg', 'utk', 'cuman', 'deh', 'Btw', 'tapi', 'gua', 'gue', 'lo', 'lu',
                  'kalo', 'trs', 'jd', 'nih', 'ntar', 'nya', 'lg', 'gk', 'ecusli', 'dpt',
                  'dr', 'kpn', 'kok', 'kyk', 'donk', 'yah', 'u', 'ya', 'ga', 'km', 'eh',
                  'sih', 'eh', 'bang', 'br', 'kyk', 'rp', 'jt', 'kan', 'gpp', 'sm', 'usah',
                  'mas', 'sob', 'thx', 'ato', 'jg', 'gw', 'wkwk', 'mak', 'haha', 'iy', 'k',
                  'tp', 'haha', 'dg', 'dri', 'duh', 'ye', 'wkwkwk', 'syg', 'btw',
                  'nerjemahin', 'gaes', 'guys', 'moga', 'kmrn', 'nemu', 'yukkk',
                  'wkwkw', 'klas', 'iw', 'ew', 'lho', 'sbnry', 'org', 'gtu', 'bwt',
                  'klrga', 'clau', 'lbh', 'cpet', 'ku', 'wke', 'mba', 'mas', 'sdh', 'kmrn',
                  'oi', 'spt', 'dlm', 'bs', 'krn', 'jgn', 'sapa', 'spt', 'sh', 'wakakaka',
                  'sihhh', 'hehe', 'ih', 'dgn', 'la', 'kl', 'ttg', 'mana', 'kmna', 'kmn',
                  'tdk', 'tuh', 'dah', 'kek', 'ko', 'pls', 'bbrp', 'pd', 'mah', 'dhhh',
                  'kpd', 'tuh', 'kzl', 'byar', 'si', 'sii', 'cm', 'sy', 'hahahaha', 'weh']

data = stop_factory + more_stopwords

dictionary = ArrayDictionary(data)
str = StopWordRemover(dictionary)
print(data)

#sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tokenize
from nltk.tokenize import TweetTokenizer

#menghapus emmoticons
emoticons_happy = set(['x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
                       ':-)', ':)', ';)', ':o)', ':]', ':3'])
emoticons_sad = set([ ':-/', '>:/', ':S', '>:[', ':@', ':-(',
                     ':[', ':-||', '=L', ':<', ':(', '>.<', ":'-("])

emoticons = emoticons_happy.union(emoticons_sad)

def clean_tweets(tweet):
  tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
  tweet_tokens = tokenizer.tokenize(tweet)

  tweets_clean = []
  for word in tweet_tokens:
    if (word not in data and word not in emoticons and word not in string.punctuation):
      stem_word = stemmer.stem(word)
      tweets_clean.append(stem_word)
  return tweets_clean

tweet_df['Tweet'] = tweet_df['remove_hastag'].apply(lambda x: clean_tweets(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['a', 'ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'arti', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'b', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagainamakah', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasannya', 'bahwasanya', 'baik', 'baiklah', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapala

In [ ]:
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat...",SobatDamai Toleransi menjadi dasar yang kuat u...,SobatDamai Toleransi menjadi dasar yang kuat u...,"[sobatdamai, toleransi, dasar, kuat, erat, sat..."
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks...",Gas bu jangan dibiarin apapun alasanya maksa i...,Gas bu jangan dibiarin apapun alasanya maksa i...,"[gas, bu, dibiarin, apa, alasanya, maksa, udah..."
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,"[penting, balik, cocok, iman, seru, apa, topik..."
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,"[beda, alir, agama, terima, industri, kreatif,..."
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ...","Sejak Negara ini belum berdiri, Perbedaan sdh ...",Sejak Negara ini belum berdiri Perbedaan sdh m...,Sejak Negara ini belum berdiri Perbedaan sdh m...,"[negara, diri, beda, lekat, bangsa, beda, lang..."


In [ ]:
def remove_punct(text):
  text  = " ".join([char for char in text if char not in string.punctuation])
  return text

tweet_df['Tweet'] = tweet_df['Tweet'].apply(lambda x: remove_punct(x))
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat...",SobatDamai Toleransi menjadi dasar yang kuat u...,SobatDamai Toleransi menjadi dasar yang kuat u...,sobatdamai toleransi dasar kuat erat satu bang...
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks...",Gas bu jangan dibiarin apapun alasanya maksa i...,Gas bu jangan dibiarin apapun alasanya maksa i...,gas bu dibiarin apa alasanya maksa udah bener ...
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,penting balik cocok iman seru apa topik bahas ...
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,beda alir agama terima industri kreatif indone...
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ...","Sejak Negara ini belum berdiri, Perbedaan sdh ...",Sejak Negara ini belum berdiri Perbedaan sdh m...,Sejak Negara ini belum berdiri Perbedaan sdh m...,negara diri beda lekat bangsa beda langgar nor...


In [ ]:
tweet_df = tweet_df[tweet_df['Tweet']!='']
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat...",SobatDamai Toleransi menjadi dasar yang kuat u...,SobatDamai Toleransi menjadi dasar yang kuat u...,sobatdamai toleransi dasar kuat erat satu bang...
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks...",Gas bu jangan dibiarin apapun alasanya maksa i...,Gas bu jangan dibiarin apapun alasanya maksa i...,gas bu dibiarin apa alasanya maksa udah bener ...
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,penting balik cocok iman seru apa topik bahas ...
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,beda alir agama terima industri kreatif indone...
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ...","Sejak Negara ini belum berdiri, Perbedaan sdh ...",Sejak Negara ini belum berdiri Perbedaan sdh m...,Sejak Negara ini belum berdiri Perbedaan sdh m...,negara diri beda lekat bangsa beda langgar nor...


In [ ]:
tweet_df = tweet_df.reset_index(drop=True)
tweet_df.head()

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat...",SobatDamai Toleransi menjadi dasar yang kuat u...,SobatDamai Toleransi menjadi dasar yang kuat u...,sobatdamai toleransi dasar kuat erat satu bang...
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks...",Gas bu jangan dibiarin apapun alasanya maksa i...,Gas bu jangan dibiarin apapun alasanya maksa i...,gas bu dibiarin apa alasanya maksa udah bener ...
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,penting balik cocok iman seru apa topik bahas ...
3,IrwanHardiant0,2022-05-31 07:00:01,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,Perbedaan dalam berbagai hal termasuk aliran d...,beda alir agama terima industri kreatif indone...
4,Prawitadisastra,2022-05-31 05:48:16,"Sejak Negara ini belum berdiri, Perbedaan sdh ...","Sejak Negara ini belum berdiri, Perbedaan sdh ...",Sejak Negara ini belum berdiri Perbedaan sdh m...,Sejak Negara ini belum berdiri Perbedaan sdh m...,negara diri beda lekat bangsa beda langgar nor...


In [ ]:
tweet_df.drop_duplicates(subset ="remove_hastag", keep = 'first', inplace = True)
tweet_df.head(3)

,username,tweetcreatedts,text,clean_tweet,remove_http,remove_hastag,Tweet
0,dutadamaijatim,2022-05-31 09:53:59,"#SobatDamai, Toleransi menjadi dasar yang kuat...","#SobatDamai, Toleransi menjadi dasar yang kuat...",SobatDamai Toleransi menjadi dasar yang kuat u...,SobatDamai Toleransi menjadi dasar yang kuat u...,sobatdamai toleransi dasar kuat erat satu bang...
1,Terajana313,2022-05-31 09:28:46,"@BuKasun_Ndeso Gas bu, jangan dibiarin, apapun...","Gas bu, jangan dibiarin, apapun alasanya maks...",Gas bu jangan dibiarin apapun alasanya maksa i...,Gas bu jangan dibiarin apapun alasanya maksa i...,gas bu dibiarin apa alasanya maksa udah bener ...
2,Funnyangelinn,2022-05-31 07:50:39,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,Dan yang terpenting dibalik semua kecocokan ad...,penting balik cocok iman seru apa topik bahas ...


In [ ]:
tweet_preprocesing = tweet_df

In [ ]:
tweet_df.drop(tweet_df.columns[[0,1,2,3,4,5]], axis = 1, inplace = True)
tweet_df.head()

,Tweet
0,sobatdamai toleransi dasar kuat erat satu bang...
1,gas bu dibiarin apa alasanya maksa udah bener ...
2,penting balik cocok iman seru apa topik bahas ...
3,beda alir agama terima industri kreatif indone...
4,negara diri beda lekat bangsa beda langgar nor...


In [ ]:
tweet_df.to_csv('data_bersih_fix.csv',encoding='utf8', index=False)

In [ ]:
!pip3 install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Importing library yang dibutuhkan
import pandas as pd
import googletrans
from googletrans import Translator

In [ ]:
df = tweet_df
df

NameError: ignored

In [ ]:
translator = Translator()
translations = {}
for column in df.columns:
    # Unique elements of the column
    unique_elements = df[column].unique()
    for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
        translations[element] = translator.translate(element).text
translations

{'afrika tampung etnis ragam budaya kaya minyak sumber daya alam benua milik posisi strategis': 'Africa accommodates ethnic diversity, rich culture, oil, natural resources, continents, belongs to a strategic position',
 'agama dunia adlh benteng umat manusia buat jahat dosa asih jaga beda provokasi dalil agama tuju pribadi adlh agama': "world religion is mankind's bastion for evil, sin, love, keep different, provocation, religious arguments, personal goals, is religion",
 'agama islam beda jenis tuntun tuntun wajib sunnah anjur klu hitung dosa': 'Islam has different types of guidance, obligatory guidance, sunnah, recommended if you count sins',
 'agree wz muslim tk islam sd smakatolik prnh minoritas dsana harga beda doktrin ajar cinta kasih bangga almamater katolik agama msh islam': 'agree wz muslim Kindergarten islam sd smakatolik prnh minority there the price is different the doctrine of teaching love love proud alma mater catholic religion still islam',
 'aja diskusi nyokap perihal 

In [ ]:
df.replace(translations, inplace = True)
df.head(10)

,Tweet
0,"Friends, peace, tolerance, strong base, close ..."
1,"What's the reason, ma'am, if you leave gas, wh..."
2,It's important that it's appropriate to have f...
3,Different religions accept Indonesia's creativ...
4,"The country itself is different, the nation is..."
5,"Fortunately, God gave Hafiz Hafizah's best fri..."
6,"Different culture, ethnicity, religion, East L..."
7,"Characteristics of moderate religion, tawassut..."
8,"yess way of live ideology embraces Islam, whic..."
9,"Guys, different religions live thanks for the day"


In [ ]:
df.to_csv('data_fix_bersih_eng.csv')